In [1]:
from pyspark.sql import SparkSession
import os


In [2]:
spark = SparkSession.builder \
        .appName("app") \
        .master("local[60]") \
        .config("spark.driver.memory", "900g") \
        .config("spark.executor.memory", "900g") \
        .config("spark.jars", "postgresql-42.3.3.jar") \
        .getOrCreate()
#spark.sparkContext.setLogLevel("DEBUG")

23/11/09 21:34:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
spark.sparkContext.setLogLevel("WARN") # ALL, DEBUG, WARN,
#spark.sparkContext.setLogLevel("ALL") # ALL, DEBUG, WARN,

In [3]:
username = os.environ.get('USERNAME', 'lsqb')
password = os.environ.get('PASSWORD', 'lsqb')
dbname = os.environ.get('DBNAME', 'lsqb')
dbhost = os.environ.get('DBHOST', 'postgres')

df_tables = spark.read.format("jdbc") \
    .option("url", f'jdbc:postgresql://{dbhost}:5432/{dbname}') \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "information_schema.tables") \
    .option("user", username) \
    .option("password", password) \
    .load()

for idx, row in df_tables.toPandas().iterrows():
        if row.table_schema == 'public':
            table_name = row.table_name
            df = spark.read.format("jdbc") \
                .option("url", f'jdbc:postgresql://{dbhost}:5432/{dbname}') \
                .option("driver", "org.postgresql.Driver") \
                .option("dbtable", table_name) \
                .option("user", username) \
                .option("password", password) \
                .load()
    
            print(table_name)
            #print(df.show())
            df.createOrReplaceTempView(table_name)

tagclass
forum
comment
post
person
comment_hastag_tag
company
university
continent
country
city
tag
post_hastag_tag
forum_hasmember_person
forum_hastag_tag
person_hasinterest_tag
person_likes_comment
person_likes_post
person_studyat_university
person_workat_company
person_knows_person
message
comment_replyof_message
message_hascreator_person
message_hastag_tag
message_islocatedin_country
person_likes_message


In [ ]:
spark.sql("SET spark.sql.yannakakis.countGroupInLeaves = false").show()

In [ ]:
spark.sql("SET spark.sql.yannakakis.enabled = false").show()
spark.sql("SET spark.sql.yannakakis.enabled = true").show()

In [ ]:
spark.sql("SET spark.sql.yannakakis.enabled = false").show()

In [ ]:
spark.sql("SET spark.sql.yannakakis.enabled = true").show()
spark.sql("SET spark.sql.yannakakis.countGroupInLeaves = false").show()

In [ ]:
spark.sql("SET spark.sql.cbo.enabled = true").show()

In [ ]:
spark.sql("ANALYZE TABLE part COMPUTE STATISTICS;").show()

In [ ]:
df = spark.sql("""select count(*) from patents p1, patents p2, patents p3 where p1.toNode = p2.fromNode AND p2.toNode = p3.fromNode""")
df.show()

df.explain(True)

In [ ]:
df = spark.sql("""select avg(i_current_price) from item, inventory where i_item_sk = inv_item_sk""")
df = spark.sql("""select avg(ss_quantity) from store_sales,store where s_store_sk = ss_store_sk""")
#df = spark.sql("""select avg(ss_list_price) from store_sales,store where s_store_sk = ss_store_sk""")
#df = spark.sql("""select count(isnull(ss_quantity)) from store_sales,store where s_store_sk = ss_store_sk""")
#df = spark.sql("""select count(isnull(i_current_price)) from item, inventory where i_item_sk = inv_item_sk""")
#df = spark.sql("""select sum(if (isnull(ss_quantity), 0, ss_quantity)) from store_sales,store where s_store_sk = ss_store_sk""")

df.show()

df.explain(True)

In [4]:
def run_query(file):
    with open(file, 'r') as f:
        query = '\n'.join(filter(lambda line: not line.startswith('limit') and not line.startswith('-'), f.readlines()))
        
        print("running query: \n" + query)
        return spark.sql(query)

In [ ]:
df = spark.sql("""
SELECT ps_partkey, count(*) from part, partsupp
WHERE p_partkey = ps_partkey
GROUP BY ps_partkey
""")

df.show()

df.explain(True)

In [ ]:
df = spark.sql("SELECT COUNT(p_size) / COUNT(DISTINCT p_size) FROM part")
df.show()
df = spark.sql("SELECT COUNT(p_retailprice) / COUNT(DISTINCT p_retailprice) FROM part")
df.show()

In [ ]:
df_t1 = spark.createDataFrame([(1,1), (2,1), (2,2), (3,2), (3,3), (4,3), (4,3), (5,2), (5,1), (6,4)], schema=("a","b"))
df_t1.createOrReplaceTempView("t1")
df_t2 = spark.createDataFrame([(1,1), (2,1), (3,2), (3,2), (3,3), (3,3), (4,3), (4,2), (5,1), (6,4)], schema=("c","d"))
df_t2.createOrReplaceTempView("t2")
df_t3 = spark.createDataFrame([(1,1), (2,1), (3,2), (3,2), (3,3), (3,3), (4,3), (4,2), (5,1), (6,4)], schema=("e","f"))
df_t3.createOrReplaceTempView("t3")

query = "select median(a) from t1, t2 where b = c"
#query = "select percentile(a, 0.5, b) from t1, t2 where b = c"
#query = "select median(a) from t1 where EXISTS (SELECT 1 FROM t2 WHERE b = c)"
#query = "select count(*) from t1, t2 where b = c"
#query = "select *a from t1 where EXISTS (SELECT 1 FROM t2 WHERE b = c)"

spark.sql("SET spark.sql.yannakakis.enabled = false").show()

df = spark.sql(query)
df.show()

spark.sql("SET spark.sql.yannakakis.enabled = true").show()

df = spark.sql(query)
df.show()

In [ ]:
spark.conf.set("spark.sql.legacy.setCommandRejectsSparkCoreConfs","false")
#spark.conf.set("spark.executor.cores", "6")
#spark.conf.set("spark.executor.instances", "6")
spark.conf.set("spark.sql.shuffle.partitions", "6")

In [5]:
import pandas as pd
import time

def benchmark(query):    
    spark.sql("SET spark.sql.yannakakis.enabled = true").show()

    start_time = time.time()

    df1 = run_query(query)
    df1.show()
    #df1.explain(True)

    end_time = time.time()
    yannakakis_time = end_time - start_time

    spark.sql("SET spark.sql.yannakakis.enabled = false").show()

    start_time = time.time()

    df2 = run_query(query)
    df2.show()
    #df2.explain(True)

    end_time = time.time()
    ref_time = end_time - start_time
    
    #return [query, ref_time, yannakakis_time]
    return [query, ref_time, yannakakis_time]


#queries = ['lsqb/sql/q1.sql', 'lsqb/sql/q3.sql', 'lsqb/sql/q4.sql']
queries = ['lsqb/sql/q1.sql', 'lsqb/sql/q4.sql'] * 1

results = [benchmark(q) for q in queries]

df = pd.DataFrame(results, columns = ['query', 'ref_time', 'yannakakis_time'])

print(df)

df.to_csv("results.csv")
    

#print(f'row count: {df1.count()} vs. {df2.count()}' )
    #print(f'time ref: {ref_time}\ntime yannakakis: {yannakakis_time}')

+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.yannaka...| true|
+--------------------+-----+

running query: 
SELECT count(*)

FROM Country

JOIN City

  ON City.isPartOf_CountryId = Country.CountryId

JOIN Person

  ON Person.isLocatedIn_CityId = City.CityId

JOIN Forum_hasMember_Person

  ON Forum_hasMember_Person.PersonId = Person.PersonId

JOIN Forum

  ON Forum.ForumId = Forum_hasMember_Person.ForumId

JOIN Post

  ON Post.Forum_containerOfId = Forum.ForumId

JOIN Comment

  ON Comment.replyOf_PostId = Post.PostId

JOIN Comment_hasTag_Tag

  ON Comment_hasTag_Tag.CommentId = Comment.CommentId

JOIN Tag

  ON Tag.TagId = Comment_hasTag_Tag.TagId

JOIN TagClass

  ON Tag.hasType_TagClassId = TagClass.TagClassId;



23/11/09 21:34:38 WARN RewriteJoinsAsSemijoins: applying yannakakis rewriting to join: Aggregate [toprettystring(count(1), Some(GMT)) AS toprettystring(count(1))#161]
+- Project
   +- Join Inner, (hasType_TagClassId#77L = TagClassId#24L)
      :- Project [hastype_tagclassid#77L]
      :  +- Join Inner, (TagId#76L = TagId#55L)
      :     :- Project [tagid#55L]
      :     :  +- Join Inner, (CommentId#54L = CommentId#32L)
      :     :     :- Project [commentid#32L]
      :     :     :  +- Join Inner, (replyOf_PostId#35L = PostId#42L)
      :     :     :     :- Project [postid#42L]
      :     :     :     :  +- Join Inner, (Forum_containerOfId#44L = ForumId#28L)
      :     :     :     :     :- Project [forumid#28L]
      :     :     :     :     :  +- Join Inner, (ForumId#28L = ForumId#84L)
      :     :     :     :     :     :- Project [forumid#84L]
      :     :     :     :     :     :  +- Join Inner, (PersonId#85L = PersonId#50L)
      :     :     :     :     :     :     :- Project [

+-----------+
|   count(1)|
+-----------+
|10492206756|
+-----------+

+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.yannaka...|false|
+--------------------+-----+

running query: 
SELECT count(*)

FROM Country

JOIN City

  ON City.isPartOf_CountryId = Country.CountryId

JOIN Person

  ON Person.isLocatedIn_CityId = City.CityId

JOIN Forum_hasMember_Person

  ON Forum_hasMember_Person.PersonId = Person.PersonId

JOIN Forum

  ON Forum.ForumId = Forum_hasMember_Person.ForumId

JOIN Post

  ON Post.Forum_containerOfId = Forum.ForumId

JOIN Comment

  ON Comment.replyOf_PostId = Post.PostId

JOIN Comment_hasTag_Tag

  ON Comment_hasTag_Tag.CommentId = Comment.CommentId

JOIN Tag

  ON Tag.TagId = Comment_hasTag_Tag.TagId

JOIN TagClass

  ON Tag.hasType_TagClassId = TagClass.TagClassId;



23/11/09 21:43:08 WARN RewriteJoinsAsSemijoins: applying yannakakis rewriting to join: Aggregate [toprettystring(count(1), Some(GMT)) AS toprettystring(count(1))#461]
+- Project
   +- Join Inner, (MessageId#135L = MessageId#126L)
      :- Project [messageid#126L]
      :  +- Join Inner, (ParentMessageId#119L = MessageId#126L)
      :     :- Project [messageid#126L]
      :     :  +- Join Inner, (MessageId#126L = MessageId#122L)
      :     :     :- Project [messageid#126L]
      :     :     :  +- Filter isnotnull(MessageId#126L)
      :     :     :     +- Relation [messageid#126L,tagid#127L] JDBCRelation(message_hastag_tag) [numPartitions=1]
      :     :     +- Project [messageid#122L]
      :     :        +- Filter isnotnull(MessageId#122L)
      :     :           +- Relation [messageid#122L,hascreator_personid#123L] JDBCRelation(message_hascreator_person) [numPartitions=1]
      :     +- Project [parentmessageid#119L]
      :        +- Filter isnotnull(ParentMessageId#119L)
      : 

+-----------+
|   count(1)|
+-----------+
|10492206756|
+-----------+

+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.yannaka...| true|
+--------------------+-----+

running query: 
SELECT count(*)

FROM Message_hasTag_Tag

JOIN Message_hasCreator_Person

  ON Message_hasTag_Tag.MessageId = Message_hasCreator_Person.MessageId

JOIN Comment_replyOf_Message 

  ON Comment_replyOf_Message.ParentMessageId = Message_hasTag_Tag.MessageId

JOIN Person_likes_Message

  ON Person_likes_Message.MessageId = Message_hasTag_Tag.MessageId;



+---------+
| count(1)|
+---------+
|654535635|
+---------+

+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.yannaka...|false|
+--------------------+-----+

running query: 
SELECT count(*)

FROM Message_hasTag_Tag

JOIN Message_hasCreator_Person

  ON Message_hasTag_Tag.MessageId = Message_hasCreator_Person.MessageId

JOIN Comment_replyOf_Message 

  ON Comment_replyOf_Message.ParentMessageId = Message_hasTag_Tag.MessageId

JOIN Person_likes_Message

  ON Person_likes_Message.MessageId = Message_hasTag_Tag.MessageId;



+---------+
| count(1)|
+---------+
|654535635|
+---------+

             query    ref_time  yannakakis_time
0  lsqb/sql/q1.sql  417.716112        93.064554
1  lsqb/sql/q4.sql   89.968812       169.508008


In [ ]:
#spark.conf.set("spark.sql.legacy.setCommandRejectsSparkCoreConfs","false")
#spark.conf.set("spark.executor.cores", "1")
#spark.conf.set("spark.executor.instances", "1")
spark.conf.set("spark.sql.shuffle.partitions", "1")

In [6]:
## Compare result
import time
#query = 'tpch-kit/dbgen/queries/postgres/2.sql'
#query = 'tpch-kit/dbgen/queries/postgres/13.sql'
#query = 'count-3.sql'
#query = 'tpch-kit/dbgen/queries/postgres/11.sql'
#query = '11-simple.sql'
query = 'median-2-hint.sql'
#query = 'tpch-kit/dbgen/queries/postgres/7.sql'
#query = '13-simple.sql'
query = 'dsb-queries/agg_queries/query101.sql'
query = 'snap-queries/patents02.sql'
#query = 'lsqb/sql/q1.sql'
query = 'lsqb/sql/q1.sql'

spark.sql("SET spark.sql.yannakakis.enabled = true").show()

start_time = time.time()

df1 = run_query(query)
df1.show()
#df1.explain(mode="extended")

end_time = time.time()
yannakakis_time = end_time - start_time

spark.sql("SET spark.sql.yannakakis.enabled = false").show()

start_time = time.time()

df2 = run_query(query)
df2.show()
#df2.explain(mode="extended")

end_time = time.time()
ref_time = end_time - start_time

#print(f'row count: {df1.count()} vs. {df2.count()}' )
print(f'time ref: {ref_time}\ntime yannakakis: {yannakakis_time}')

+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.yannaka...| true|
+--------------------+-----+

running query: 
SELECT count(*)

FROM Country

JOIN City

  ON City.isPartOf_CountryId = Country.CountryId

JOIN Person

  ON Person.isLocatedIn_CityId = City.CityId

JOIN Forum_hasMember_Person

  ON Forum_hasMember_Person.PersonId = Person.PersonId

JOIN Forum

  ON Forum.ForumId = Forum_hasMember_Person.ForumId

JOIN Post

  ON Post.Forum_containerOfId = Forum.ForumId

JOIN Comment

  ON Comment.replyOf_PostId = Post.PostId

JOIN Comment_hasTag_Tag

  ON Comment_hasTag_Tag.CommentId = Comment.CommentId

JOIN Tag

  ON Tag.TagId = Comment_hasTag_Tag.TagId

JOIN TagClass

  ON Tag.hasType_TagClassId = TagClass.TagClassId;



23/11/09 18:35:43 WARN RewriteJoinsAsSemijoins: applying yannakakis rewriting to join: Aggregate [toprettystring(count(1), Some(GMT)) AS toprettystring(count(1))#243]
+- Project
   +- Join Inner, (hasType_TagClassId#65L = TagClassId#68L)
      :- Project [hastype_tagclassid#65L]
      :  +- Join Inner, (TagId#64L = TagId#99L)
      :     :- Project [tagid#99L]
      :     :  +- Join Inner, (CommentId#98L = CommentId#76L)
      :     :     :- Project [commentid#76L]
      :     :     :  +- Join Inner, (replyOf_PostId#79L = PostId#86L)
      :     :     :     :- Project [postid#86L]
      :     :     :     :  +- Join Inner, (Forum_containerOfId#88L = ForumId#72L)
      :     :     :     :     :- Project [forumid#72L]
      :     :     :     :     :  +- Join Inner, (ForumId#72L = ForumId#106L)
      :     :     :     :     :     :- Project [forumid#106L]
      :     :     :     :     :     :  +- Join Inner, (PersonId#107L = PersonId#94L)
      :     :     :     :     :     :     :- Projec

+----------+
|  count(1)|
+----------+
|3113376721|
+----------+

+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.yannaka...|false|
+--------------------+-----+

running query: 
SELECT count(*)

FROM Country

JOIN City

  ON City.isPartOf_CountryId = Country.CountryId

JOIN Person

  ON Person.isLocatedIn_CityId = City.CityId

JOIN Forum_hasMember_Person

  ON Forum_hasMember_Person.PersonId = Person.PersonId

JOIN Forum

  ON Forum.ForumId = Forum_hasMember_Person.ForumId

JOIN Post

  ON Post.Forum_containerOfId = Forum.ForumId

JOIN Comment

  ON Comment.replyOf_PostId = Post.PostId

JOIN Comment_hasTag_Tag

  ON Comment_hasTag_Tag.CommentId = Comment.CommentId

JOIN Tag

  ON Tag.TagId = Comment_hasTag_Tag.TagId

JOIN TagClass

  ON Tag.hasType_TagClassId = TagClass.TagClassId;



+----------+
|  count(1)|
+----------+
|3113376721|
+----------+

time ref: 113.04056096076965
time yannakakis: 27.786081075668335
